# Ejemplo de modelos realistas


Los valores revisados en la bibliografía y los ejemplos simulados se muestran en detalle en nuestro trabajo:

https://doi.org/10.1088/2057-1976/ab3330

Son cortes de muñeca y de tobillo. Pueden tener cuatro tipos de tejidos, cuyas propiedades dieléctricas en $f=1300$ MHz son: 

- *Skin wet*, variable entre $\varepsilon_r = [28.0, 46.8]$ y $\sigma = [1.00, 1.08]$ S/m y un espesor variable entre $[1.46, 2.88]$ mm.
- *Muscle* variable entre $\varepsilon_r = [29.0, 70.0]$ y  $\sigma = [0.90, 1.55]$ S/m. 
- *Cortical bone* fijo $\varepsilon_r =12.1$ y  $\sigma =  0.20$ S/m, espesor variable entre $[0.50, 3.00]$ mm
- *Trabecular bone* variable entre $\varepsilon_r = [12.5, 20.1]$ y  $\sigma =  [0.44, 0.92]$ S/m. 

El medio de acoplamiento puede ser agua destilada y sal, es la solución estudiada en este trabajo (https://doi.org/10.1118/1.4788640), en particular es la solución *salt 17.5* cuyas propiedades dieléctricas relativas son: $ɛ^{*}_r≈ 76.95-26.97j$.


## Para correr en Google Colab

Ejecutar este documento en forma dinámica:[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://github.com/rirastorza/heelSimulationDB/blob/main/examples/ejemplo_realistas2D.ipynb)

In [4]:
%%capture
%%capture  
try:
    import dolfin
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenics-install-real.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
    import dolfin

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,518 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,859 kB]
Get:13 https://ppa.launchpadcontent.net/deadsnak

'1.17.1'

In [5]:
%%capture
!wget https://raw.githubusercontent.com/rirastorza/heelSimulationDB/blob/main/meshes/pacient10_munieca_slice1.geo


## Ejemplo de uso: simulación con FDTD

### 1. Grafico de un corte de muñeca

